# Power electricity power consumption prediction model

Let's first start by importing the needed libraries

In [1]:
from pyspark.sql import SparkSession

And create a local parallel spark session

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("Power electricity prediction") \
        .getOrCreate()

The we read the dataframe we need to apply the prediction on

In [3]:
train = spark.read.csv('../data/engineered/powerelectricity_train.csv', header=True, inferSchema=True)
test = spark.read.csv('../data/engineered/powerelectricity_test.csv', header=True, inferSchema=True)

Now we will test some regression models and evaluate them using rmse

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor, LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

In [5]:
train_ep = train.drop('rms_current')
train_ep = train_ep.drop('electric_power')

#train_rms = train.drop('electric_power')

test_ep = test.drop('rms_current')
test_ep = test_ep.drop('electric_power')

#test_rms = test.drop('electric_power')

features_ep = train_ep.columns.copy()
#features_ep.remove('electric_power')
features_ep.remove('delta_y')

#features_rms = train_rms.columns.copy()
#features_rms.remove('rms_current')

In [6]:
assembler_pe_train = VectorAssembler(inputCols=features_ep,outputCol="features")
train_ep = assembler_pe_train.transform(train_ep)
assembler_pe_test = VectorAssembler(inputCols=features_ep,outputCol="features")
test_ep = assembler_pe_test.transform(test_ep)

Let's start by predicting electric_power

In [231]:
#rf_pe = RandomForestRegressor(featuresCol="features",labelCol='delta_y', maxDepth=15, maxBins=14, minInstancesPerNode=1, minInfoGain=0.0005, subsamplingRate=1, seed=42, numTrees=50)
#rf_pe = RandomForestRegressor(featuresCol="features",labelCol='delta_y', maxDepth=6,numTrees=20,seed=12345)
rf_pe = GBTRegressor(featuresCol="features",labelCol='delta_y',seed=12345)

In [ ]:
%%time

model_pe = rf_pe.fit(train_ep)

In [ ]:
predictions = model_pe.transform(train_ep)

In [ ]:
print('################################ TRAIN ################################')
evaluator_rmse = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on train data = %g" % rmse)

evaluator_r2 = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions)
print("R squared (r2) on train data = %g" % r2)

evaluator_mae = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="mae")
mae = evaluator_mae.evaluate(predictions)
print("Mean Average Error (RMSE) on train data = %g" % mae)

In [ ]:
predictions = model_pe.transform(test_ep)

In [ ]:
print('################################ TEST ################################')

evaluator_rmse = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator_r2 = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions)
print("R squared (r2) on test data = %g" % r2)

evaluator_mae = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="mae")
mae = evaluator_mae.evaluate(predictions)
print("Mean Average Error (RMSE) on test data = %g" % mae)

In [ ]:
predictions.select(["delta_y", "prediction"]).show(100000)

And finally save to a file the predictions

In [ ]:
import os

try:
    os.mkdir('../predictions/')
except:
    pass

preds = predictions.select(["prediction"])
preds.toPandas().to_csv('../predictions/preds.csv')